Jack Tillman
Physics 5300 Final Project

Gravitational Orbits Notebook

Let's first outline the algorithm we'll use to solve this problem numerically. 

The force from 1 body acting on another is given by Newton's Law of Universal Gravitation:
$$
F = G \frac{m_1 m_2}{|\overrightarrow{\Delta r}|^3} \overrightarrow{\Delta r}
$$

With this, we can create Cartesian definitions for $\overrightarrow{\Delta r}$ in terms of x, y, & z:
$$
\overrightarrow{r_1} = x_1 \hat{x} + y_1 \hat{y} + z_1 \hat{z} \\
\overrightarrow{r_2} = x_2 \hat{x} + y_2 \hat{y} + z_2 \hat{z}
$$

$\overrightarrow{\Delta r}$ is then just the difference between $\overrightarrow{r_2}$ & $\overrightarrow{r_1}$:

$\overrightarrow{\Delta r} = (x_2 - x_1) \hat{x} + (y_2 - y_1) \hat{y} + (z_2 - z_1) \hat{z}$


Now that we have the force defined in terms of Cartesian coordinates, we can seperate it into each component and numerically integrate accordingly. As such, we'll have to integrate the following set of 6 coupled differential equations. 

For ease of writing, we'll define $|\overrightarrow{\Delta r}|$ before writing out each equation. In doing so, we recognize that the coupling of the equations is found in this term. The 6 equations to numerically integrate are:
$$
\\
\overrightarrow{\Delta r} = (x_2 - x_1) \hat{x} + (y_2 - y_1) \hat{y} + (z_2 - z_1) \hat{z} \\ \\
$$

The differential equations are then: 

$$
\ddot{x_1} = G \frac{m_2}{|\overrightarrow{\Delta r}|^3} (x_2 - x_1) \\ 
$$
$$
\ddot{y_1} = G \frac{m_2}{|\overrightarrow{\Delta r}|^3} (y_2 - y_1) \\ 
$$
$$
\ddot{z_1} = G \frac{m_2}{|\overrightarrow{\Delta r}|^3} (z_2 - z_1) \\ 
$$
$$
\ddot{x_2} = -G \frac{m_1}{|\overrightarrow{\Delta r}|^3} (x_2 - x_1) \\ 
$$
$$
\ddot{y_2} = -G \frac{m_1}{|\overrightarrow{\Delta r}|^3} (y_2 - y_1) \\ 
$$
$$
\ddot{z_2} = -G \frac{m_1}{|\overrightarrow{\Delta r}|^3} (z_2 - z_1) \\ 
$$

After numerically integrating these equations, and supplementing some initial conditions, we'll have solved for the two vectors as functions of time, $\overrightarrow{r_1}$ & $\overrightarrow{r_2}$. These will be functions of time for which we can visualize the orbits and analyze further if needed. 

Before jumping into implementing the algorithm, we can note that by setting one mass to be much larger than the other, we can demonstrate that the problem reduces to the orbits seen in class. Furthermore, with appropriate settings of constants, initial conditions and masses, we can even visualize orbits for planets in our solar system, such as the Sun and Earth!

One last comment: We'll apply the Runge - Kutta algorithm as our numerical integrator. In doing so, we'll be wary of the fact that energy is not conserved, although it performs better than the Euler method of numerical integration. 

Let's now implement this algorithm and see where it takes us:

Code used in this file was written by Jack Tillman, with multiple sections applying code written by Professor Furnstahl in "Orbital_eqns_with_different_algorithms.ipynb" & "Orbit_games.ipynb"

In [2]:
#Importing all necessary python libraries:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.integrate import solve_ivp

In [3]:
#Creating a class that we can use to solve for any orbit we'll consider
class OrbitSolver:
    def __init__(self, G=1., m1 = 1., m2 = 1.):
        #This initializes the value of the gravitational constant and the masses of the 2 bodies
        self.G = G
        self.m1 = m1
        self.m2 = m2

    def d_dt(self, t, compVector):
        #This initializes a vector that returns the right hand side of the differential equations mentioned
        #in the markdowns above

        """
        This function initializes a vector that returns the right hand side of the differential equation in the 
        markdowns above

        Parameters:
        t : float
            time
        
        compVector: float
                    12 component vector with the following format
                        compVector[0] = x1(t) & compVector[1] = x1'(t)
                        compVector[2] = y1(t) & compVector[3] = y1'(t)
                        compVector[4] = z1(t) & compVector[5] = z1'(t)
                        compVector[6] = x2(t) & compVector[7] = x2'(t)
                        compVector[8] = y2(t) & compVector[9] = y2'(t)
                        compVector[10] = z2(t) & compVector[11] = z2'(t)
        
        Returns
        ---------
        
        """
        deltaR = np.sqrt( (compVector[6] - compVector[0])**2 + (compVector[8] - compVector[2])**2 + (compVector[10] - compVector[4])**2)
        return [ \
                compVector[1], self.G * self.m2 * (compVector[6] - compVector[0]) / deltaR**3, \
                compVector[3], self.G * self.m2 * (compVector[8] - compVector[2]) / deltaR**3, \
                compVector[5], self.G * self.m2 * (compVector[10] - compVector[4]) / deltaR**3, \
                compVector[7], -self.G * self.m1 * (compVector[6] - compVector[0]) / deltaR**3, \
                compVector[9], -self.G * self.m1 * (compVector[8] - compVector[2]) / deltaR**3, \
                compVector[11], -self.G * self.m1 * (compVector[10] - compVector[4]) / deltaR**3, \
                ]

    def solve_ode(self, time_pts, initConditions, abserr=1.0e-9, relerr=1.0e-9):
        #This function will solve (numerically integrate) our differential equations based on the Runge - Kutta 
        #algorithm along with the array of time points we construct and our initial conditions

        solution = solve_ivp(self.d_dt, (time_pts[0], time_pts[-1]), initConditions, t_eval=time_pts, method='RK23', 
                             atoll=abserr, rtol=relerr)
        x1, x1Dot, y1, y1Dot, z1, z1Dot, x2, x2Dot, y2, y2Dot, z2, z2Dot = solution.y
        
        return x1, x1Dot, y1, y1Dot, z1, z1Dot, x2, x2Dot, y2, y2Dot, z2, z2Dot

In [4]:
#Let's now create initial conditions and use our OrbitSolver() class

#Creating our time 'vector'
tStart = 0.0
tEnd = 10.0
deltaT = 0.001
time_pts = np.arange(tStart, tEnd, deltaT)

#Initializing our constants and masses:
G = 1.0
m1 = 1.0
m2 = 5.0

#Initializing an orbit via our OrbitSolver() class:
orbit1 = OrbitSolver(G, m1, m2)

#Creating our initial conditions
x1_0 = 1.0 
x1Dot_0 = 1.0

y1_0 = 1.0
y1Dot_0 = -1.0

z1_0 = 0.0
z1Dot_0 = -0.25

x2_0 = -(m1 / m2) * x1_0
x2Dot_0 = -(m1 / m2) * x1Dot_0

y2_0 = -(m1 / m2) * y1_0
y2Dot_0 = -(m1 / m2) * y1Dot_0

z2_0 = -(m1 / m2) * z1_0
z2Dot_0 = -(m1 / m2) * z1Dot_0

initConditions = [x1_0, x1Dot_0, y1_0, y1Dot_0, z1_0, z1Dot_0, 
                  x2_0, x2Dot_0, y2_0, y2Dot_0, z2_0, z2Dot_0]

#Solving for x1(t), y1(t), z1(t), x2(t), y2(t), z2(t):
x1, x1Dot, y1, y1Dot, z1, z1Dot, x2, x2Dot, y2, y2Dot, z2, z2Dot = orbit1.solve_ode(time_pts, initConditions)

orbit1Vector = [x1, x1Dot, y1, y1Dot, z1, z1Dot, x2, x2Dot, y2, y2Dot, z2, z2Dot]

m1Orbit = [orbit1Vector[0], orbit1Vector[2], orbit1Vector[4]]
m2Orbit = [orbit1Vector[6], orbit1Vector[8], orbit1Vector[10]]

c:\Users\Jaydu\anaconda3\lib\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `atoll`.
  warn("The following arguments have no effect for a chosen solver: {}."


In [5]:
#Plotting the current orbit on a scatter plot to visualize it's accuracy
fig1 = plt.figure(1, figsize = (16, 9))
title1 = 'Gravitational Orbit: ' + \
                rf' $m_1 = {m1:.1f},$' + \
                rf' $m_2 = {m2:.1f},$' + \
                rf' $G = {G:.1f}$' 


ax1 = plt.axes(projection = '3d')
ax1.scatter3D(m1Orbit[0], m1Orbit[1], m1Orbit[2], color = 'red', marker = '.', s = 5.00)
ax1.scatter3D(m2Orbit[0], m2Orbit[1], m2Orbit[2], color = 'blue', marker = '.', s = 5.00)

plt.title(title1, fontweight = 'bold')
ax1.set_xlabel('X Position')
ax1.set_ylabel('Y Position')
ax1.set_zlabel('Z Position')

ax1.legend([rf' $m_1$ Orbit', rf'$m_2$ Orbit'], loc = 'upper right')

plt.tight_layout()
plt.savefig('Grav_Orbit_Similar_Masses.png')
plt.clf()

<Figure size 1600x900 with 0 Axes>

Now that we've coded a solver for the differential equations we encounter, our next task is to demonstrate that the 2 body problem reduces when 1 mass is much larger than the other, and we're in it's frame of reference. To do this, we'll demonstrate that the velocity of the larger mass tends to 0 as its mass increases.

To demonstrate this, we'll plot the orbit from earlier, this time increasing the mass of the larger object for each orbit. We'll then plot all orbits on a single figure to allow for comparison.  

In [6]:
#Here, we'll use the same initial conditions as before. As such, we have no need to update them here
largerMassVector = [5, 10, 50, 100, 500, 1000]
initX1Dot = [1.0, 2.0, 5.0, 7.0, 10.0, 15.0]
initY1Dot = [-1.0, -2.0, -5.0, -7.0, -10.0, -15.0]

mass1OrbitData = []
mass2OrbitData = []

tStart = 0.0
tEnd = 10.0
deltaT = 0.001
time_pts = np.arange(tStart, tEnd, deltaT)

i = 0
for i in range( len(largerMassVector) ):
    mLarger = largerMassVector[i]
    orbitLM = OrbitSolver(G, m1, mLarger)

    #Creating our initial conditions
    x1_0 = 1.0 
    x1Dot_0 = initX1Dot[i]

    y1_0 = 1.0
    y1Dot_0 = initY1Dot[i]

    z1_0 = 0.0
    z1Dot_0 = 0.0

    x2_0 = -(m1 / mLarger) * x1_0
    x2Dot_0 = -(m1 / mLarger) * x1Dot_0

    y2_0 = -(m1 / mLarger) * y1_0
    y2Dot_0 = -(m1 / mLarger) * y1Dot_0

    z2_0 = -(m1 / mLarger) * z1_0
    z2Dot_0 = -(m1 / mLarger) * z1Dot_0

    initConditions = [x1_0, x1Dot_0, y1_0, y1Dot_0, z1_0, z1Dot_0, 
                    x2_0, x2Dot_0, y2_0, y2Dot_0, z2_0, z2Dot_0]
    
    x1, x1Dot, y1, y1Dot, z1, z1Dot, x2, x2Dot, y2, y2Dot, z2, z2Dot = orbitLM.solve_ode(time_pts, initConditions)
    orbitLMVector = [x1, x1Dot, y1, y1Dot, z1, z1Dot, x2, x2Dot, y2, y2Dot, z2, z2Dot]

    mass1OrbitData.append([orbitLMVector[0], orbitLMVector[2], orbitLMVector[4]])
    mass2OrbitData.append([orbitLMVector[6], orbitLMVector[8], orbitLMVector[10]])

#We can now plot the orbits as subplots on a single figure to visualize how the problem reduces when one mass
#is much larger than the other

fig2 = plt.figure(figsize = (16, 9))
overallTitle = 'Gravitational Orbits with Varying Masses'

fig2.suptitle(overallTitle, fontweight = 'bold')

ax1 = fig2.add_subplot(2, 3, 1)
ax1.scatter(mass1OrbitData[0][0], mass1OrbitData[0][1], color = 'red', marker = '.', s = 5.00)
ax1.scatter(mass2OrbitData[0][0], mass2OrbitData[0][1], color = 'blue', marker = '.', s = 5.00)
ax1.set_title(rf' $G = {G},$' + rf' $m_1 = {m1:.1f},$' + rf' $m_2 = {largerMassVector[0]:.1f}$')
ax1.set_xlabel('X Position')
ax1.set_ylabel('Y Position')
ax1.legend([rf' $m_1$ Orbit', rf'$m_2$ Orbit'], loc = 'upper right')

ax2 = fig2.add_subplot(2, 3, 2)
ax2.scatter(mass1OrbitData[1][0], mass1OrbitData[1][1], color = 'red', marker = '.', s = 5.00)
ax2.scatter(mass2OrbitData[1][0], mass2OrbitData[1][1], color = 'blue', marker = '.', s = 5.00)
ax2.set_title(rf' $G = {G},$' + rf' $m_1 = {m1:.1f},$' + rf' $m_2 = {largerMassVector[1]:.1f}$')
ax2.set_xlabel('X Position')
ax2.set_ylabel('Y Position')
ax2.legend([rf' $m_1$ Orbit', rf'$m_2$ Orbit'], loc = 'upper right')

ax3 = fig2.add_subplot(2, 3, 3)
ax3.scatter(mass1OrbitData[2][0], mass1OrbitData[2][1], color = 'red', marker = '.', s = 5.00)
ax3.scatter(mass2OrbitData[2][0], mass2OrbitData[2][1], color = 'blue', marker = '.', s = 5.00)
ax3.set_title(rf' $G = {G},$' + rf' $m_1 = {m1:.1f},$' + rf' $m_2 = {largerMassVector[2]:.1f}$')
ax3.set_xlabel('X Position')
ax3.set_ylabel('Y Position')
ax3.legend([rf' $m_1$ Orbit', rf'$m_2$ Orbit'], loc = 'upper right')

ax4 = fig2.add_subplot(2, 3, 4)
ax4.scatter(mass1OrbitData[3][0], mass1OrbitData[3][1], color = 'red', marker = '.', s = 5.00)
ax4.scatter(mass2OrbitData[3][0], mass2OrbitData[3][1], color = 'blue', marker = '.', s = 5.00)
ax4.set_title(rf' $G = {G},$' + rf' $m_1 = {m1:.1f},$' + rf' $m_2 = {largerMassVector[3]:.1f}$')
ax4.set_xlabel('X Position')
ax4.set_ylabel('Y Position')
ax4.legend([rf' $m_1$ Orbit', rf'$m_2$ Orbit'], loc = 'upper right')

ax5 = fig2.add_subplot(2, 3, 5)
ax5.scatter(mass1OrbitData[4][0], mass1OrbitData[4][1], color = 'red', marker = '.', s = 5.00)
ax5.scatter(mass2OrbitData[4][0], mass2OrbitData[4][1], color = 'blue', marker = '.', s = 5.00)
ax5.set_title(rf' $G = {G},$' + rf' $m_1 = {m1:.1f},$' + rf' $m_2 = {largerMassVector[4]:.1f}$')
ax5.set_xlabel('X Position')
ax5.set_ylabel('Y Position')
ax5.legend([rf' $m_1$ Orbit', rf'$m_2$ Orbit'], loc = 'upper right')

ax6 = fig2.add_subplot(2, 3, 6)
ax6.scatter(mass1OrbitData[5][0], mass1OrbitData[5][1], color = 'red', marker = '.', s = 5.00)
ax6.scatter(mass2OrbitData[5][0], mass2OrbitData[5][1], color = 'blue', marker = '.', s = 5.00)
ax6.set_title(rf' $G = {G},$' + rf' $m_1 = {m1:.1f},$' + rf' $m_2 = {largerMassVector[5]:.1f}$')
ax6.set_xlabel('X Position')
ax6.set_ylabel('Y Position')
ax6.legend([rf' $m_1$ Orbit', rf'$m_2$ Orbit'], loc = 'upper right')

fig2.tight_layout()
fig2.savefig('Grav_Orbit_Varying_Masses.png')
plt.clf()


<Figure size 1600x900 with 0 Axes>

Here, we can see that the orbit of the larger mass continuously decreases in size as it's mass increases. When the mass is large enough, it essentially acts as a point and does not orbit whatsoever. This is the result we've seen in class! 

When the larger mass's orbit is so minute that it can be ignored, we're able to treat the two body problem as a single body one. In this case, the larger mass only contributes a potential energy to the system rather than contributing any motion of it's own. 

We can now focus on implementing the Leapfrog Algorithm as a means of solving for the orbit of a particle. This algorithm conserves energy, whereas the numerical integration performed earlier does not. As such, we'll be able to demonstrate this fact via creating a plot. 

To implement the Leapfrog Algorithm, we'll modify code written in "Orbital_eqs_with_different_algorithms.ipynb", written by Professor Furnstahl

Here, we'll create a new class which will allow us to compare the two solution algorithms. 

In [36]:
class Orbit:
    """
    Potentials and associated differential equations for central force motion
    with the potential U(r) = k r^n.  Several algorithms for integration of 
    ordinary differential equations are now available. 
    """
    
    def __init__(self, ang_mom, n, k=1, mu=1):
        self.ang_mom = ang_mom
        self.n = n
        self.k = k
        self.mu = mu
    
    def U(self, r):
        """Potential energy of the form U = kr^n."""
        return self.k * r**self.n
    
    def Ucf(self, r):
        """Centrifugal potential energy"""
        return self.ang_mom**2 / (2. * self.mu * r**2)
    
    def Ueff(self, r):
        """Effective potential energy"""
        return self.U(r) + self.Ucf(r)
    
    def U_deriv(self, r):
        """dU/dr"""
        return self.n * self.k * r**(self.n - 1)
        
    def Ucf_deriv(self, r):
        """dU_cf/dr"""
        return -2. * self.ang_mom**2 / (2. * self.mu * r**3)
        
    def Ueff_deriv(self, r):
        """dU_eff/dr"""
        return self.U_deriv(r) + self.Ucf_deriv(r)
        
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dr/dt d^2r/dt^2 dphi/dt]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            3-component vector with y[0] = r(t), y[1] = dr/dt, y[2] = phi
            
        """
        return [ y[1], 
                -1./self.mu * self.Ueff_deriv(y[0]), 
                self.ang_mom / (self.mu * y[0]**2) ]
    
    
    def solve_ode(self, t_pts, r_0, r_dot_0, phi_0,
                  method='RK23',
                  abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Use solve_ivp with the option of specifying the method.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [r_0, r_dot_0, phi_0]  
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, method=method, 
                             atol=abserr, rtol=relerr)
        r, r_dot, phi = solution.y
        return r, r_dot, phi
    
    def solve_ode_Euler(self, t_pts, r_0, r_dot_0, phi_0):
        """
        Solve the ODE given initial conditions with the Euler method.
        The accuracy is determined by the spacing of times in t_pts.
        """
        
        delta_t = t_pts[1] - t_pts[0]
        
        # initialize the arrays for r, rdot, phi with zeros
        num_t_pts = len(t_pts)    # length of the array t_pts
        r = np.zeros(num_t_pts)
        r_dot = np.zeros(num_t_pts)
        phi = np.zeros(num_t_pts)
        
        # initial conditions
        r[0] = r_0
        r_dot[0] = r_dot_0
        phi[0] = phi_0
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            y = [r[i], r_dot[i], phi[i]]
            r[i+1] = r[i] + self.dy_dt(t,y)[0] * delta_t
            r_dot[i+1] = r_dot[i] + self.dy_dt(t,y)[1] * delta_t 
            phi[i+1] = phi[i] + self.dy_dt(t,y)[2] * delta_t
        return r, r_dot, phi   
    
    
    def solve_ode_Leapfrog(self, t_pts, r_0, r_dot_0, phi_0):
        """
        Solve the ODE given initial conditions with the Leapfrog method.
        """
        delta_t = t_pts[1] - t_pts[0]
        
        # initialize the arrays for r, rdot, r_dot_half, phi with zeros
        num_t_pts = len(t_pts)
        r = np.zeros(num_t_pts)
        r_dot = np.zeros(num_t_pts)
        r_dot_half = np.zeros(num_t_pts)
        phi = np.zeros(num_t_pts)
        
        # initial conditions
        r[0] = r_0
        r_dot[0] = r_dot_0
        phi[0] = phi_0
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            y = [r[i], r_dot[i], phi[i]]
            r_dot_half[i] = r_dot[i] + self.dy_dt(t, y)[1] * delta_t/2.
            r[i+1] = r[i] + r_dot_half[i] * delta_t
            
            y = [r[i+1], r_dot[i], phi[i]]
            r_dot[i+1] = r_dot_half[i] + self.dy_dt(t, y)[1] * delta_t/2.
            
            phi[i+1] = phi[i] + self.dy_dt(t,y)[2] * delta_t
        return r, r_dot, phi   
        
    
    def energy(self, t_pts, r, r_dot):
        """Evaluate the energy as a function of time"""
        return (self.mu/2.) * r_dot**2 + self.Ueff(r)

def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

In [54]:
#Here we'll describe the potential energy we're considering:
n = -1.0
k = 1.0
ang_mom = 1.0

#Initializing the RK23 orbit and Leapfrog Orbit
orbitComp = Orbit(ang_mom, n, k, mu=1.)

#Initializing our time to plot over
t_start = 0.
t_end = 10.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)

#Initial conditions for both orbits:
r_0 = 1.0
r_dot_0 = 0.0
phi_0 = 0.0

In [55]:
#Now that we have both orbits established, let's actually solve each one:

#Starting with the RK23 Method:
r_ptsRK23, r_dot_ptsRK23, phi_ptsRK23 = orbitComp.solve_ode(t_pts, r_0, r_dot_0, phi_0)

#Leapfrog Method:
r_pts_LF, r_dot_pts_LF, phi_pts_LF = orbitComp.solve_ode_Leapfrog(t_pts, r_0, r_dot_0, phi_0)

#Euler Method:
r_pts_Euler, r_dot_pts_Euler, phi_pts_Euler = orbitComp.solve_ode_Euler(t_pts, r_0, r_dot_0, phi_0)
#Computing the energy & eccentricity of each orbit:
c = orbitComp.ang_mom**2 / (np.abs(orbitComp.k) * orbitComp.mu)
epsilon = c / r_0 - 1.
energy_0 = orbitComp.mu/2. * r_dot_0**2 + orbitComp.Ueff(r_0)


In [56]:
#Now that we have our orbits solved for, we need to demonstrate that the Leapfrog algorithm preserves energy 
#whereas the RK23 method of numerical integration does not

#We'll demonstrate this by creating a plot of the change in energy over time
E_tot_ptsRK23 = orbitComp.energy(t_pts, r_ptsRK23, r_dot_ptsRK23)
E_tot_0RK23 = E_tot_ptsRK23[0]
E_tot_rel_ptsRK23 = np.abs((E_tot_ptsRK23 - E_tot_0RK23)/E_tot_0RK23)

E_tot_pts_LF = orbitComp.energy(t_pts, r_pts_LF, r_dot_pts_LF)
E_tot_0_LF = E_tot_pts_LF[0]
E_tot_rel_pts_LF = np.abs((E_tot_pts_LF - E_tot_0_LF)/E_tot_0_LF)

E_tot_pts_Euler = orbitComp.energy(t_pts, r_pts_Euler, r_dot_pts_Euler)
E_tot_0_Euler = E_tot_pts_Euler[0]
E_tot_rel_pts_Euler = np.abs((E_tot_pts_Euler - E_tot_0_Euler)/E_tot_0_Euler)

In [59]:
#Plotting!
fig3 = plt.figure(figsize=(6,6))

overall_title = 'Orbit:  ' + \
                rf' $n = {orbitRK23.n},$' + \
                rf' $k = {orbitRK23.k:.1f},$' + \
                rf' $l = {orbitRK23.ang_mom:.1f},$' + \
                rf' $r_0 = {r_0:.1f},$' + \
                rf' $\dot r_0 = {r_dot_0:.1f},$' + \
                rf' $\phi_0 = {phi_0:.1f}$' + \
                '\n'     # \n means a new line (adds some space here)
fig3.suptitle(overall_title, va='baseline')
ax7 = fig3.add_subplot(1,1,1)

ax7.semilogy(t_pts, E_tot_rel_ptsRK23, 
               color='green', label=r'$\Delta E(t)$ Leapfrog')
ax7.semilogy(t_pts, E_tot_rel_pts_LF, 
               color='red', label=r'$\Delta E(t)$ RK23')
ax7.semilogy(t_pts, E_tot_rel_pts_Euler, 
               color='blue', label=r'$\Delta E(t)$ Euler')

ax7.set_ylim(1.e-10, 1.e-2)    # (1.e-12, 5)
ax7.set_xlabel(r'$t$')
ax7.set_ylabel(r'Energy')
ax7.set_title('Change in energy with time')
ax7.legend()

fig3.tight_layout()
fig3.savefig('Energy_Conserv_Comparison.png', dpi=200, bbox_inches='tight')
fig3.clf()

<Figure size 600x600 with 0 Axes>

Here, we can immediately note that the Euler method of solving for the orbit produces a significant change in energy as time progresses. As such, since the energy varies with time, energy is not conserved in this solution method. 

The RK23 method performs much better than the Euler method, however, it too has a noticeable change in energy as time progresses. This solution method will deviate from the actual energy much less than the Euler method, however, it too will deviate. 

Lastly, we can see that the Leapfrog method has a change in energy that decreases as time continues. Since this is the case, it will continuously approach the actual energy of the orbit. As time were to progress, this difference would approach true zero, and the energy found in the Leapfrog method would be equivalent to the actual energy of the orbit. Since this is the case, we can conclude that the Leapfrog method preserves the energy of the orbit being solved for. 